In [3]:
import pandas as pd
import os

In [6]:
%pwd

'c:\\Users\\Aditya\\OneDrive\\Documents\\DataScience\\projects\\product-recommendation'

In [5]:
os.chdir('../')

In [7]:
data=pd.read_csv("data//flipkart_product_review.csv")
data.head()

,product_id,product_title,rating,summary,review
0,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Terrific purchase,1-more flexible2-bass is very high3-sound clar...
1,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Terrific purchase,Super sound and good looking I like that prize
2,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Super!,Very much satisfied with the device at this pr...
3,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Super!,"Nice headphone, bass was very good and sound i..."
4,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Terrific purchase,Sound quality super battery backup super quali...


In [8]:
data.columns

Index(['product_id', 'product_title', 'rating', 'summary', 'review'], dtype='object')

In [9]:
df=data[["product_title","review"]]

In [9]:
df.head()

,product_title,review
0,BoAt Rockerz 235v2 with ASAP charging Version ...,1-more flexible2-bass is very high3-sound clar...
1,BoAt Rockerz 235v2 with ASAP charging Version ...,Super sound and good looking I like that prize
2,BoAt Rockerz 235v2 with ASAP charging Version ...,Very much satisfied with the device at this pr...
3,BoAt Rockerz 235v2 with ASAP charging Version ...,"Nice headphone, bass was very good and sound i..."
4,BoAt Rockerz 235v2 with ASAP charging Version ...,Sound quality super battery backup super quali...


In [10]:
product_list=[]

## Iterate over the rows of the DataFrame

for index,row in df.iterrows():
    object={
        "product_name": row["product_title"],
        "review":row["review"]
    }

## Append the object to the product list
    product_list.append(object)

In [11]:
product_list[0]

{'product_name': 'BoAt Rockerz 235v2 with ASAP charging Version 5.0 Bluetooth Headset',
 'review': "1-more flexible2-bass is very high3-sound clarity is good 4-battery back up to 6 to 8 hour's 5-main thing is fastest charging system is available in that. Only 20 min charge and get long up to 4 hours back up 6-killing look awesome 7-for gaming that product does not support 100% if you want for gaming then I'll recommend you please don't buy but you want for only music then this product is very well for you.. 8-no more wireless headphones are comparing with that headphones at this pric..."}

In [12]:
from langchain_core.documents import Document

In [13]:
docs=[]

for object in product_list:
    metadata={"product_name":object["product_name"]}
    page_content=object["review"]

    doc=Document(page_content=page_content,metadata=metadata)
    docs.append(doc)

In [14]:
docs[1]

Document(metadata={'product_name': 'BoAt Rockerz 235v2 with ASAP charging Version 5.0 Bluetooth Headset'}, page_content='Super sound and good looking I like that prize')

In [15]:
len(docs)

450

In [16]:
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
ASTRA_DB_API_ENDPOINT = os.getenv("ASTRA_DB_API_ENDPOINT")
ASTRA_DB_APPLICATION_TOKEN = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_KEYSPACE = os.getenv("ASTRA_DB_KEYSPACE")
HF_TOKEN = os.getenv("HF_TOKEN")

In [18]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [19]:
embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\Aditya\AppData\Local\Temp\ipykernel_32992\3123569051.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\Aditya\OneDrive\Documents\DataScience\projects\product-recommendation\prodrec_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
from langchain_astradb import AstraDBVectorStore

In [21]:
vstore=AstraDBVectorStore(
    embedding=embeddings,
    collection_name="product_recommendation_db",
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
    namespace=ASTRA_DB_KEYSPACE,
)

In [22]:
insert_ids=vstore.add_documents(docs)

In [30]:
insert_ids

['cdc8388dd5a444c99139e5c216488126',
 '901b9ec1d0834f039dd434646122ad15',
 '0860fb1f934440ba8610f0ae10e53a74',
 '2bbefa54daed4a208a91e2150ac271fe',
 'd9de1687384748a7aec12f2c9dff5ed8',
 '82c0cf5bea5e4a629f0ccd354403d1c4',
 '7734f7c58d6e4e44b10d6f0c8358e516',
 '20bb8844464d44d3abb4ab961db6ed90',
 '599c6fb650fc4dd1837cf6fb18b42325',
 '3784a2fcebd6465ba476caa5152236b4',
 '70079c18d42a4bff812717fad7f1e76b',
 'f6aa40350d22432db708e29b75be3235',
 '0e13f7b2c28f4b35b50bf30b75c6ee4e',
 '8bc8acce29f04ec9a54f88c555ad3aa8',
 '27d437619e8f457a8c85beaf13a2af7a',
 '25cbb2d306e84bd281517f111d99f0c8',
 '17e0e29c07a64a068a8288089c37aef4',
 '53f2111481964ab98c3306b1f2dde194',
 '96f4727ebc9a4afca26545fee7b7e0a3',
 'faef59564ece4e0a8d310f5e2a7904dd',
 '506775aeef804b6f82a2c58e84df27b6',
 '7aa6f723388d4049b328f740ae858c77',
 '5abcb0c9381e464294143d7007f1a083',
 'befd7ef5239e4005a9ab88382df4ca2d',
 '7b344d0994ce40bca1fad1c0cdcdc70a',
 '99bcc5f34a5d4599a20b5158613c84c5',
 '28f8bbaf31204e86a5e64a6e5c4ed85e',
 

In [23]:
from langchain_groq import ChatGroq

In [24]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever

In [100]:
model=ChatGroq(groq_api_key=GROQ_API_KEY,model="llama-3.3-70b-versatile",temperature=0.6)

In [101]:
retriever_prompt = ("Given a chat history and the latest user question which may refer context in the chat history,"
    "Formulate a standalone question which can be understood without the chat history."
    "Do NOT answer the question, just reformulate it if needed and otherwise return it as is."
    )

In [102]:
retriever=vstore.as_retriever(search_kwargs={"k":3})

In [103]:
from langchain_core.prompts import ChatPromptTemplate

In [104]:
context_ques_prompt=ChatPromptTemplate.from_messages(
    [
        ("system", retriever_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human","{input}"),
    ]
)

In [105]:
history_aware_retriever=create_history_aware_retriever(model,retriever,context_ques_prompt)

In [106]:
PRODUCT_BOT_TEMPLATE="""
    You are an ecommerce bot, expert in product recommendations and responding to customer queries.
    As a bot, analyze product titles and reviews to provide accurate and helpful responses.
    Ensure your answers are relevant to the product context and refrain from straying off-topic.
    Your responses should be concise and informative

    CONTEXT:
    {context}

    QUESTION: {input}

    YOUR ANSWER:

    """

In [107]:
qa_prompt=ChatPromptTemplate.from_messages(
    [
        ("system",PRODUCT_BOT_TEMPLATE),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human","{input}")
    ]
)

In [108]:
question_answer_chain=create_stuff_documents_chain(model,qa_prompt)

In [109]:
chain=create_retrieval_chain(history_aware_retriever,question_answer_chain)

In [110]:
chat_history=[]

In [111]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [121]:
store={}

In [113]:
def get_session_history(session_id: str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

In [114]:
chain_with_memory=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [122]:
chain_with_memory.invoke(
    {"input": "can you tell me the best bluetooth buds??"},
    config={
        "configurable":{"session_id":"sess01"}
    },
)['answer']

'Based on the review, the best budget Bluetooth earphones are from One Plus, known for their superb sound quality, good bass, and very good built quality, with 15 hours of playback.'

In [123]:
store

{'sess01': InMemoryChatMessageHistory(messages=[HumanMessage(content='can you tell me the best bluetooth buds??', additional_kwargs={}, response_metadata={}), AIMessage(content='Based on the review, the best budget Bluetooth earphones are from One Plus, known for their superb sound quality, good bass, and very good built quality, with 15 hours of playback.', additional_kwargs={}, response_metadata={})])}

In [124]:
chain_with_memory.invoke(
    {"input" : "What is my previous question?"},
    config={
        "configurable":{"session_id":"sess01"}
    }
)['answer']

'Your previous question was "can you tell me the best bluetooth buds??"'

In [ ]:
store

In [125]:
chain_with_memory.invoke(
    {"input" : "What is my previous question?"},
    config={
        "configurable":{"session_id":"sess02"}
    }
)['answer']

"You didn't ask a question previously. This conversation just started, and I'm ready to help with any questions you have about the product."